# Results from MILP Realtime Computations

**NOTE:** When about to make a new run, please **duplicate** this notebook then add necessary graphs and things below.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

# Add more CTM models here as needed
from ctmmodels.ringbarrier import DTSimplexRingBarrier as NewModel
from ctmmodels.parentmodel import ParentModel as OldModel

In [7]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

**Notes on the models and parameters being used:**

* Default parameters used before:

{

    time_range = 30
    time_ticks = np.arange(0, time_range+1, 1)

    parameters = {
        'r_left': 1.0/3.0,
        'r_through': 1.0/3.0,
        'r_right': 1.0/3.0,
        'sat_flow_rate': 1800,
        'time_range': time_range,
        'time_step': 2,
        'g_min': 6,
        'g_max': 20
    }

}

* Saturation flow rate was set to 1800 vphpl, as this was the closest realistic value that matched the calculations from the parent paper.

* Traffic will be distributed evenly between the 4 lanes (left, through 1, through 2, and right). This is based on previous tests where left was getting essentially 0 green time.

* Time range will be set to 60 seconds, or 30 time steps at 2 seconds per time step.

In [8]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

# Varying parameters: demand, alpha, beta, gamma

In [9]:
def run_model(demand, weights=(0.4, 0.4, 0.2), new_model=True, log_output=True):
    if new_model:
        model = NewModel(
            demand=demand,
            alpha=weights[0],
            beta=weights[1],
            gamma=weights[2],
            **parameters
        )
    else:
        model = OldModel(
            demand=demand,
            **parameters
        )

    model.generate()
    runtime = model.solve(log_output=log_output)
    dfx, dfy, dfg = model.return_solution()
    dfparams = model.return_parameters()
    obj_values = model.return_objective_value()
    return dfx, dfy, dfg, dfparams, obj_values, runtime, model

## Intersection Cell Network

![title](img/4leg-intersection.png)

## Setting the path of cells to check

In [10]:
_approach = SOUTHBOUND

_approach_terms = [
    'Left',
    'Through',
    'Right'
]

_cell_path = [
    (CELL_SOURCE,0,_approach),
    (CELL_NORMAL,0,_approach),
    (CELL_NORMAL,1,_approach),
    (CELL_NORMAL,2,_approach),
    (CELL_MOVEMENT,THROUGH_TURN,_approach),
    S_mapping((CELL_MOVEMENT,THROUGH_TURN,_approach))[0]
]

_movement_labels = {
    (2,LEFT_TURN,NORTHBOUND): 'Northbound, Left',
    (2,RIGHT_TURN,NORTHBOUND): 'Northbound, Right',
    (2,THROUGH_TURN,NORTHBOUND): 'Northbound, Through',
    (2,LEFT_TURN,SOUTHBOUND): 'Southbound, Left',
    (2,RIGHT_TURN,SOUTHBOUND): 'Southbound, Right',
    (2,THROUGH_TURN,SOUTHBOUND): 'Southbound, Through',
    (2,LEFT_TURN,EASTBOUND): 'Eastbound, Left',
    (2,RIGHT_TURN,EASTBOUND): 'Eastbound, Right',
    (2,THROUGH_TURN,EASTBOUND): 'Eastbound, Through',
    (2,LEFT_TURN,WESTBOUND): 'Westbound, Left',
    (2,RIGHT_TURN,WESTBOUND): 'Westbound, Right',
    (2,THROUGH_TURN,WESTBOUND): 'Westbound, Through',
}

def movement_paths(approach):
    return [
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,LEFT_TURN,approach),
            S_mapping((CELL_MOVEMENT,LEFT_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,THROUGH_TURN,approach),
            S_mapping((CELL_MOVEMENT,THROUGH_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,RIGHT_TURN,approach),
            S_mapping((CELL_MOVEMENT,RIGHT_TURN,approach))[0]
        ]
    ]

## Saving dataframes

In [11]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Loading CSV files

In [15]:
df_tmp = pd.read_csv(DF_PATH + 'milp-results/results_d(450, 900)_epoch1_a0.1_b0.8_c0.1.csv')
df_tmp = df_tmp.rename(columns={
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
})

In [16]:
df_tmp

,runtime,delay,throughput,obj-value
0,2.948953,7358.873418,67.835443,349.052166


In [46]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (0.8, 0.1, 0.1),
    (0.1, 0.8, 0.1),
    (0.1, 0.1, 0.8),
    (0.45, 0.45, 0.1),
    (0.1, 0.45, 0.45),
    (0.45, 0.1, 0.45),
    (0.33, 0.33, 0.33),
    (0, 0, 0),
]

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
}

_model_type = {
    (0.8, 0.1, 0.1): 'Delay priority',
    (0.1, 0.8, 0.1): 'Throughput priority',
    (0.1, 0.1, 0.8): 'Flow priority',
    (0.45, 0.45, 0.1): 'Delay-Throughput priority',
    (0.1, 0.45, 0.45): 'Throughput-Flow priority',
    (0.45, 0.1, 0.45): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [50]:
_results_dflist = []

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_old.csv'.format(demand))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_old.csv'.format(demand))
        else:
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_a{}_b{}_c{}.csv'.format(demand, *weights))

        df_e1 = df_e1.rename(columns=_col_rename)
        df_e1['demand'] = [demand]
        df_e1['new_model'] = weights != (0,0,0)
        df_e1['alpha'] = weights[0]
        df_e1['beta'] = weights[1]
        df_e1['gamma'] = weights[2]
        df_e1['model_type'] = _model_type[weights]
        df_e1['epoch'] = 1
        
        df_e2 = df_e2.rename(columns=_col_rename)
        df_e2['demand'] = [demand]
        df_e2['new_model'] = weights != (0,0,0)
        df_e2['alpha'] = weights[0]
        df_e2['beta'] = weights[1]
        df_e2['gamma'] = weights[2]
        df_e2['model_type'] = _model_type[weights]
        df_e2['epoch'] = 2
        
        _results_dflist.append(pd.concat([df_e1, df_e2]))

In [51]:
df = pd.concat(_results_dflist)

In [53]:
save_df(df, 'milp-realtime-results')

## Generate Tables

In [56]:
# Average runtime

df_runtime_table = df.pivot_table(index='demand', columns='model_type', values='runtime', aggfunc=np.mean)
df_runtime_table

model_type,Delay priority,Delay-Flow priority,Delay-Throughput priority,Equal priority,Flow priority,Parent model,Throughput priority,Throughput-Flow priority
demand,,,,,,,,
450,2.961140,2.651713,2.843178,2.915706,3.237333,0.124217,2.873061,2.735562
900,2.158557,2.158847,2.340586,2.018723,1.500374,0.152828,1.985901,1.965661
"(450, 900)",2.658140,2.448191,3.090973,2.594204,2.397003,0.131111,2.902532,2.792279
"(900, 1800)",2.078627,1.976440,1.840762,2.057974,2.197879,0.111880,2.189268,2.082379


In [57]:
df_delay_table = df.pivot_table(index='demand', columns='model_type', values='delay', aggfunc=np.mean)
df_delay_table

model_type,Delay priority,Delay-Flow priority,Delay-Throughput priority,Equal priority,Flow priority,Parent model,Throughput priority,Throughput-Flow priority
demand,,,,,,,,
450,5826.082278,6918.702532,5384.905063,6194.177215,6243.677215,5950.832255,5440.905063,5569.987342
900,11435.613924,11127.018987,11287.297468,11339.613924,11405.113924,9883.151866,11467.113924,11372.613924
"(450, 900)",7849.386076,8019.386076,8125.107595,9229.493671,8454.873418,8515.037938,8759.911392,9154.215190
"(900, 1800)",13136.113924,13119.113924,13064.518987,13263.613924,13249.613924,12537.259457,13296.113924,13216.613924


In [58]:
df_throughput_table = df.pivot_table(index='demand', columns='model_type', values='throughput', aggfunc=np.mean)
df_throughput_table

model_type,Delay priority,Delay-Flow priority,Delay-Throughput priority,Equal priority,Flow priority,Parent model,Throughput priority,Throughput-Flow priority
demand,,,,,,,,
450,66.835443,67.835443,67.335443,68.335443,68.335443,76.253166,68.335443,68.335443
900,69.335443,69.835443,69.835443,69.835443,67.835443,80.253166,69.335443,69.335443
"(450, 900)",69.335443,68.335443,68.335443,67.835443,69.835443,78.753166,68.335443,68.335443
"(900, 1800)",69.335443,69.335443,67.835443,69.835443,69.335443,77.753166,67.335443,70.335443


## Adding initial MILP results

In [61]:
df_initial_results = pd.concat([
    pd.read_pickle(DF_PATH + 'results1.pkl'),
    pd.read_pickle(DF_PATH + 'results2.pkl')
])

In [63]:
df_initial_results.head()

,demand_ns,demand_ew,runtime,delay,throughput,objective_value,alpha,beta,gamma
0,450,450,174.118326,767.272152,51.278481,382.056221,0.80,0.10,0.10
1,450,450,415.136684,791.607595,55.000000,-291.456345,0.10,0.80,0.10
2,450,450,891.892373,801.107595,54.000000,-1585.763682,0.10,0.10,0.80
3,450,450,343.920306,767.272152,51.278481,49.987495,0.45,0.45,0.10
4,450,450,596.610098,791.607595,55.000000,-937.523394,0.10,0.45,0.45


In [65]:
df.head()

,runtime,delay,throughput,obj-value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,3.216363,4455.658228,66.835443,3167.632535,450,True,0.8,0.1,0.1,Delay priority,1
0,2.705916,7196.506329,66.835443,5233.862720,450,True,0.8,0.1,0.1,Delay priority,2
0,2.739941,4326.658228,67.835443,46.513092,450,True,0.1,0.8,0.1,Throughput priority,1
0,3.006180,6555.151899,68.835443,278.233317,450,True,0.1,0.8,0.1,Throughput priority,2
0,3.362435,4997.658228,67.835443,-775.294246,450,True,0.1,0.1,0.8,Flow priority,1


In [76]:
df_initial_results['demand'] = df_initial_results.apply(lambda row: 
                                                        row['demand_ns'] if row['demand_ns'] == row['demand_ew']
                                                        else (int(row['demand_ns']), int(row['demand_ew']))
                                                        , axis=1)

df_initial_results['new_model'] = df_initial_results.apply(lambda row: row['alpha'] != 0, axis=1)

df_initial_results['model_type'] = df_initial_results.apply(lambda row: _model_type[
    (row['alpha'], row['beta'], row['gamma'])
], axis=1)

df_initial_results['epoch'] = 0

In [79]:
df_initial_results = df_initial_results[[
    'runtime',
    'delay',
    'throughput',
    'objective_value',
    'demand',
    'new_model',
    'alpha',
    'beta',
    'gamma',
    'model_type',
    'epoch'
]]

In [85]:
df = df.rename(columns={'obj-value': 'objective_value'})

In [88]:
df_final = pd.concat([df_initial_results, df]).sort_values(by=['epoch', 'demand', 'model_type'])

In [89]:
df_final

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,174.118326,767.272152,51.278481,382.056221,450,True,0.80,0.10,0.10,Delay priority,0
5,456.304917,767.272152,51.278481,-598.035046,450,True,0.45,0.10,0.45,Delay-Flow priority,0
3,343.920306,767.272152,51.278481,49.987495,450,True,0.45,0.45,0.10,Delay-Throughput priority,0
6,917.078324,767.272152,51.278481,-487.789937,450,True,0.33,0.33,0.33,Equal priority,0
2,891.892373,801.107595,54.000000,-1585.763682,450,True,0.10,0.10,0.80,Flow priority,0
0,0.227479,743.113924,66.835443,66.835443,450,False,0.00,0.00,0.00,Parent model,0
1,415.136684,791.607595,55.000000,-291.456345,450,True,0.10,0.80,0.10,Throughput priority,0
4,596.610098,791.607595,55.000000,-937.523394,450,True,0.10,0.45,0.45,Throughput-Flow priority,0
1,5.594564,2080.658228,60.000000,1209.951345,900,True,0.80,0.10,0.10,Delay priority,0
6,6.865374,2080.658152,60.000003,-643.922642,900,True,0.45,0.10,0.45,Delay-Flow priority,0
